<a href="https://colab.research.google.com/github/hmingkwan/Assessment/blob/main/Manulife_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogen langchain llama_index

In [ ]:
!pip install dask[dataframe]

In [ ]:
!pip install accelerate

In [ ]:
!pip install unsloth

In [ ]:
import yfinance as yf
from transformers import AutoModelForCausalLM, AutoTokenizer,  TextStreamer
from autogen import Agent
import requests
import torch
from unsloth import FastLanguageModel


In [ ]:
# Load the Meta LLaMA 3.1 8B model and tokenizer
model_name = "unsloth/Llama-3.2-1B-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-3B-Instruct"
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

In [16]:
model = FastLanguageModel.for_inference(model)

In [17]:
# Define Data Analyst Agent
class DataAnalystAgent(Agent):
    def __init__(self, model, tokenizer, device):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.device = device

    def analyze_data(self, client_info):
        # Analyze client information and generate insights
        prompt = f"Analyze the following client information and provide a financial advice report:\n{client_info}"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(**inputs, max_new_tokens=500)
        insights = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return insights

In [18]:
# Define Recommendation Agent
class RecommendationAgent(Agent):
    def __init__(self, model, tokenizer, data_analyst, device):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.data_analyst = data_analyst
        self.device = device

    def generate_recommendation(self, client_info):
        insights = self.data_analyst.analyze_data(client_info)
        prompt = f"Generate a recommendation for the following client: {client_info} based on the following insights: {insights}"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(**inputs, max_new_tokens=500)
        recommendation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return recommendation

In [19]:
# Define Reporting Agent
class ReportingAgent:
    def __init__(self):
        pass

    def generate_report(self, client_info, insights, recommendations):
        report = f"Client Financial Report\n"
        report += f"========================\n"
        report += f"Client Information:\n{client_info}\n\n"
        report += f"Insights:\n{insights}\n\n"
        report += f"Recommendations:\n{recommendations}\n"
        report += f"========================\n"
        report += f"End of Report"
        return report

In [21]:
# client information
client_info1 = """
Client Information:
- Name: John Chan
- Age: 35
- Occupation: Software Engineer
- Email: john.chan@abc.com
- Annual Income: HKD $780,000
- Monthly Expenses: HKD $45,000
- Monthly Savings: HKD $15,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $100,000 (60% in stocks, 40% in bonds)
  - Stock Portfolio: HKD $600,000 (Tech stocks: $400,000, Diversified ETFs: $200,000)
- Debt:
  - Mortgage: HKD $2,000,000 (5% interest)
  - Car Loan: HKD $100,000 (3% interest)
- Financial Goals:
  - Short-Term: Build an emergency fund of HKD $700,000 within 1 year.
  - Medium-Term: Save HKD $1,000,000 for a down payment on a house in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $10,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%

"""

In [22]:
client_info2 = """
Client Information:
- Name: Emily Wong
- Age: 28
- Occupation: Marketing Specialist
- Email: emily.wong@xyz.com
- Annual Income: HKD $480,000
- Monthly Expenses: HKD $30,000
- Monthly Savings: HKD $20,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $50,000 (70% in stocks, 30% in bonds)
  - Stock Portfolio: HKD $150,000 (Tech stocks: $100,000, Diversified ETFs: $50,000)
- Debt:
  - Student Loan: HKD $200,000 (4% interest)
- Financial Goals:
  - Short-Term: Save HKD $100,000 for travel within 1 year.
  - Medium-Term: Save HKD $500,000 for a home down payment in 3 years.
  - Long-Term: Build a retirement fund of HKD $5,000,000 by age 60.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%

"""

In [23]:
client_info3 = """
Client Information:
- Name: Robert Lee
- Age: 55
- Occupation: Financial Consultant
- Email: robert.lee@abc.com
- Annual Income: HKD $1,200,000
- Monthly Expenses: HKD $70,000
- Monthly Savings: HKD $30,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $300,000 (50% in stocks, 50% in bonds)
  - Stock Portfolio: HKD $1,200,000 (Blue-chip stocks: $800,000, Diversified ETFs: $400,000)
- Debt:
  - Mortgage: HKD $1,500,000 (3.5% interest)
- Financial Goals:
  - Short-Term: Pay off mortgage within 5 years.
  - Medium-Term: Save HKD $2,000,000 for retirement in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $20,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%
"""

In [25]:
# Define workflow
def workflow(client_info, model, tokenizer, device):
    # Initialize agents
    data_analyst = DataAnalystAgent(model, tokenizer, device)
    recommendation_agent = RecommendationAgent(model, tokenizer, data_analyst, device)
    reporting_agent = ReportingAgent()

    # Step 1: Analyze client data
    insights = data_analyst.analyze_data(client_info)

    # Step 2: Generate recommendations based on insights
    recommendations = recommendation_agent.generate_recommendation(client_info)

    # Step 3: Generate a report
    report = reporting_agent.generate_report(client_info, insights, recommendations)

    return {
        "insights": insights,
        "recommendations": recommendations,
        "report": report,
    }

In [ ]:
# Run workflow for client 1
result1 = workflow(client_info1, model, tokenizer, device)
# print(result1)

In [31]:
formatted_report1 = result1['report'].replace('\\n', '\n')
print(formatted_report1)

Client Financial Report
Client Information:

Client Information:
- Name: John Chan
- Age: 35
- Occupation: Software Engineer
- Email: john.chan@abc.com
- Annual Income: HKD $780,000
- Monthly Expenses: HKD $45,000
- Monthly Savings: HKD $15,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $100,000 (60% in stocks, 40% in bonds)
  - Stock Portfolio: HKD $600,000 (Tech stocks: $400,000, Diversified ETFs: $200,000)
- Debt:
  - Mortgage: HKD $2,000,000 (5% interest)
  - Car Loan: HKD $100,000 (3% interest)
- Financial Goals:
  - Short-Term: Build an emergency fund of HKD $700,000 within 1 year.
  - Medium-Term: Save HKD $1,000,000 for a down payment on a house in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $10,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%



Insights:
Analyze the following client information and provide a financial advice report:

Client Information:
- Name: John Chan
- Age: 35
- Occupation: Software Eng

In [36]:
# Run workflow for client 2
result2 = workflow(client_info2, model, tokenizer, device)

In [37]:
formatted_report2 = result2['report'].replace('\\n', '\n')
print(formatted_report2)

Client Financial Report
Client Information:

Client Information:
- Name: Emily Wong
- Age: 28
- Occupation: Marketing Specialist
- Email: emily.wong@xyz.com
- Annual Income: HKD $480,000
- Monthly Expenses: HKD $30,000
- Monthly Savings: HKD $20,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $50,000 (70% in stocks, 30% in bonds)
  - Stock Portfolio: HKD $150,000 (Tech stocks: $100,000, Diversified ETFs: $50,000)
- Debt:
  - Student Loan: HKD $200,000 (4% interest)
- Financial Goals:
  - Short-Term: Save HKD $100,000 for travel within 1 year.
  - Medium-Term: Save HKD $500,000 for a home down payment in 3 years.
  - Long-Term: Build a retirement fund of HKD $5,000,000 by age 60.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%



Insights:
Analyze the following client information and provide a financial advice report:

Client Information:
- Name: Emily Wong
- Age: 28
- Occupation: Marketing Specialist
- Email: emily.wong@xyz.com
- Annual Income: HK

In [38]:
# Run workflow for client 3
result3 = workflow(client_info3, model, tokenizer, device)

In [39]:
formatted_report3 = result3['report'].replace('\\n', '\n')
print(formatted_report3)

Client Financial Report
Client Information:

Client Information:
- Name: Robert Lee
- Age: 55
- Occupation: Financial Consultant
- Email: robert.lee@abc.com
- Annual Income: HKD $1,200,000
- Monthly Expenses: HKD $70,000
- Monthly Savings: HKD $30,000
- Investments:
  - Mandatory Provident Fund (MPF): HKD $300,000 (50% in stocks, 50% in bonds)
  - Stock Portfolio: HKD $1,200,000 (Blue-chip stocks: $800,000, Diversified ETFs: $400,000)
- Debt:
  - Mortgage: HKD $1,500,000 (3.5% interest)
- Financial Goals:
  - Short-Term: Pay off mortgage within 5 years.
  - Medium-Term: Save HKD $2,000,000 for retirement in 5 years.
  - Long-Term: Retire at age 60 with a monthly income of HKD $20,000.
- Assumptions:
- Inflation Rate: 2.5%
- Interest Rate: 3.0%
- Tax Rate: 15%


Insights:
Analyze the following client information and provide a financial advice report:

Client Information:
- Name: Robert Lee
- Age: 55
- Occupation: Financial Consultant
- Email: robert.lee@abc.com
- Annual Income: HKD $1,2